In [1]:
import numpy as np                          # Basic Array and Numaric Operation
import os                                   # use to access the files 
import tarfile                              # use to extract dataset from zip files
import sys
import zipfile                              # to extract zip file


import torch                                # Provides basic tensor operation and nn operation
import torchvision                          # Provides facilities to access image dataset


import my_utils.loadDataset as dl           # create dataloader for selected dataset
import my_utils.loadModel as lm             # facilitate loading and manipulating models
import my_utils.trainModel as tm            # Facilitate training of the model
import my_utils.initialize_pruning as ip    # Initialize and provide basic parmeter require for pruning
import my_utils.facilitate_pruning as fp    # Compute Pruning Value and many things

### Prepare dataloader and set image properties. Check for cuda device and load model

In [2]:
# set the locationn of the dataset and trai and test data folder name
dl.setFolderLocation(datasets       ='/home/pragnesh/Dataset/',
                     selectedDataset='IntelIC/',
                     train          ='train',
                     test           ='test')
# set the imge properties
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

#load the saved model if have any
load_path = "/home/pragnesh/Model/Model/VGG_IntelIC_v2"
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
newModel = torch.load(load_path, map_location=torch.device('cpu'))

#if dont have any saved trained model download pretrained model for tranfer learning
# newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
#                         freeze_feature=False,device_l=device1)

#print(newModel)

### Here we have 3 set of parameter.One for data loader, second for model and third are hyperparameter
### Train the model

In [ ]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(#set locations of the dataset, train and test data
                 dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 # Selecat a variant of VGGNet
                 ModelName='vgg16',model=newModel,device_l=device1,
                 # Set all the Hyper-Parameter for training
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)

#Save the  trained model 
SavePath = '/home/pragnesh/Model/vgg16-v2'
torch.save(newModel, SavePath)

### Pruning Initialization

In [3]:
blockList   = ip.createBlockList(newModel)              #ip.getBlockList('vgg16')
#featureList = ip.createFeatureList(newModel)
convIdx     = ip.createFeatureList(newModel)
module = ip.getPruneModule(newModel)
print(f"Block List  = {blockList}\n"
      f"Conv Index  = {convIdx}\n"
     )
for i in range(len(module)):
    print(f"{i+1} :{module[i]}")

Block List  = [2, 2, 3, 3, 3]
Conv Index  = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]

1 :Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
2 :Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
3 :Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
4 :Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
5 :Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
6 :Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
7 :Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
8 :Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
9 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
10 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
12 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
13 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), p

### Pruning Process Starts From Here
##### get candidate convolution pruning layer

In [4]:
candidateConvLayer = []
st =0
en = 4
for i in range(st,en):
    candidateConvLayer.append(fp.compute_distance_score(module[i]._parameters['weight'],threshold=1))
print("\n\n\nHere is the :",len(candidateConvLayer))


Shape of the tensor: torch.Size([64, 3, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 3])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 3
...
Shape of the tensor: torch.Size([64, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 64])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 64
................................................................
Shape of the tensor: torch.Size([128, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([128, 64])
Number Of Features Map in current  layer l     = 128
Number Of Features Map in previous layer (l-1) = 64
................................................................
Shape of the tensor: torch.Size([128, 128, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([128, 128])
Number Of Features Map in current  layer l     = 128
Number Of Fe

In [5]:
print("Channe list :",len(candidateConvLayer))
for i in range(3):
    print("Number of input channel :",len(candidateConvLayer[i]))
    for j in range(3):
        print(f"Number of kernel pair with simirities diff less than threshold {i} :{len(candidateConvLayer[i][j])}")
        for k in range(3):
            print("Channe list [] [] [] :",(candidateConvLayer[i][j][k]))

Channe list : 4
Number of input channel : 3
Number of kernel pair with simirities diff less than threshold 0 :190
Channe list [] [] [] : [0, 0, 15, tensor(0.9576)]
Channe list [] [] [] : [0, 0, 19, tensor(0.9328)]
Channe list [] [] [] : [0, 0, 22, tensor(0.9021)]
Number of kernel pair with simirities diff less than threshold 0 :195
Channe list [] [] [] : [1, 0, 15, tensor(0.9780)]
Channe list [] [] [] : [1, 0, 17, tensor(0.9487)]
Channe list [] [] [] : [1, 0, 28, tensor(0.9798)]
Number of kernel pair with simirities diff less than threshold 0 :200
Channe list [] [] [] : [2, 0, 33, tensor(0.9690)]
Channe list [] [] [] : [2, 1, 3, tensor(0.8423)]
Channe list [] [] [] : [2, 1, 18, tensor(0.9357)]
Number of input channel : 64
Number of kernel pair with simirities diff less than threshold 1 :176
Channe list [] [] [] : [0, 0, 1, tensor(0.9870)]
Channe list [] [] [] : [0, 0, 2, tensor(0.8020)]
Channe list [] [] [] : [0, 0, 16, tensor(0.8548)]
Number of kernel pair with simirities diff less th

In [6]:
for i in range(len(candidateConvLayer)):
    fp.sort_kernel_by_distance(candidateConvLayer[i])

In [7]:
print(f'c={len(candidateConvLayer)}, c[] ={len(candidateConvLayer[0])}, c[][] = {len(candidateConvLayer[0][0])},c[][][] = {len(candidateConvLayer[0][0][0])}')

c=4, c[] =3, c[][] = 190,c[][][] = 4


In [8]:
def get_k_element(channel_list,k):
    channel_k_list = []
    for i in range(len(channel_list)):
        tempList = []
        for j in range(k):
            tempList.append(channel_list[i][j])
        channel_k_list.append(tempList)
    return channel_k_list

In [9]:
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=0.1,)
newList = []
for i in range(len(candidateConvLayer)):
    newList.append(fp.get_k_element(channel_list=candidateConvLayer[i],k=5)  )


In [10]:
print(f'c={len(newList)}, c[] ={len(newList[1])}, c[][] = {len(newList[1][0])},c[][][] = {len(newList[1][0][0])}')

c=4, c[] =64, c[][] = 5,c[][][] = 4


In [11]:
print(len(newList),end=' ');
for i in range(1):
    print(len(newList[i]),end=' ');
    for j in range(1):
        print(len(newList[i][j]));
        print(newList[0][0])

4 3 5
[[0, 19, 22, tensor(0.4883)], [0, 48, 55, tensor(0.5158)], [0, 4, 38, tensor(0.5298)], [0, 34, 49, tensor(0.5376)], [0, 21, 47, tensor(0.5471)]]


In [12]:
for i in range(len(newList)):
    for j in range(5):
        print(newList[i][j])

[[0, 19, 22, tensor(0.4883)], [0, 48, 55, tensor(0.5158)], [0, 4, 38, tensor(0.5298)], [0, 34, 49, tensor(0.5376)], [0, 21, 47, tensor(0.5471)]]
[[1, 13, 61, tensor(0.4906)], [1, 25, 26, tensor(0.5661)], [1, 1, 25, tensor(0.5776)], [1, 39, 58, tensor(0.6031)], [1, 15, 58, tensor(0.6107)]]
[[2, 50, 63, tensor(0.5478)], [2, 24, 58, tensor(0.5731)], [2, 5, 13, tensor(0.6033)], [2, 22, 54, tensor(0.6091)], [2, 8, 31, tensor(0.6269)]]


IndexError: list index out of range

### Now Execute the pruning

#### Define class to execute custom pruning

In [ ]:
import torch.nn.utils.prune as prune
class KernalPruningMethod(prune.BasePruningMethod):
    PRUNING_TYPE = 'unstructured'
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        #mask.view(-1)[::2] = 0
        size = t.shape
        print(size)
        for k1 in range(len(newList)):
            for k2 in range(len(newList[k1])):
                i= newList[k1][k2][1]
                j= newList[k1][k2][0]
                
                print(f"i= {i} , j= {j}")
                
                mask[i][j] = 0
        return mask
def kernal_unstructured(module, name):
    KernalPruningMethod.apply(module, name)
    return module

### Pass the cadidate layer of Conv Net for custom pruning

In [ ]:
print(len(newList))
print(len(newList[0]))

In [ ]:
layer_number = 0
for i in range(3):
    kernal_unstructured(module[i], name = 'weight')
    layer_number += 1

In [ ]:
print(module)